# Predict image from model

## Import Lib

In [81]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from PIL import Image

## Load model and input

In [91]:
model = keras.models.load_model("/content/drive/MyDrive/ai_prj/prj_Image Super-Resolution/model/SRCNN_div2k.h5")
model.load_weights("./content/drive/MyDrive/ai_prj/prj_Image Super-Resolution/checkpoint/srcnn/cp.h5")

ValueError: ignored

In [ ]:
def prepare_img(path):
# path = "/content/drive/MyDrive/ai_prj/Cinque_Terre_.jpeg"
  lr = np.array(Image.open(path).convert("RGB"))
  image = tf.expand_dims(lr, axis=0)
  image = tf.cast(image, tf.float32)
  sr = model(image)
  sr = tf.clip_by_value(sr, 0, 255)
  sr = tf.round(sr)
  sr = tf.cast(sr, tf.uint8)
  sr = np.array(sr)
  sr = np.squeeze(sr,axis=0)

## Prepare input

In [ ]:
# Random crop image
def rd_crop(lr_img, hr_img, hr_crop_size=96, scale=4):

    lr_crop_size = hr_crop_size // scale
    lr_img_shape = tf.shape(lr_img)[:2]

    lr_width = tf.random.uniform(shape=(), maxval=lr_img_shape[1] - lr_crop_size + 1, dtype=tf.int32)
    lr_height = tf.random.uniform(shape=(), maxval=lr_img_shape[0] - lr_crop_size + 1, dtype=tf.int32)

    hr_height = lr_height * scale
    hr_width = lr_width * scale


    lr_img_cropped = lr_img[lr_height:lr_height + lr_crop_size, lr_width:lr_width + lr_crop_size]
    hr_img_cropped = hr_img[hr_height:hr_height + hr_crop_size, hr_width:hr_width + hr_crop_size]

    iloc = np.array([[(lr_width,lr_height),(lr_width + lr_crop_size, lr_height+lr_crop_size)],[(hr_width,hr_height),(hr_width + hr_crop_size,hr_height+hr_crop_size)]])

    return lr_img_cropped, hr_img_cropped, iloc

## Plot result

In [ ]:
import cv2


lr_c, sr_c , iloc= rd_crop(lr,sr,hr_crop_size=100)

#fullsize
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
out_lr = cv2.rectangle(lr.copy(),tuple(iloc[0,0]),tuple(iloc[0,1]),(255, 0, 0),2)
plt.imshow(out_lr)
plt.subplot(1,2,2)
out_sr = cv2.rectangle(sr.copy(),tuple(iloc[1,0]),tuple(iloc[1,1]),(255, 0, 0),8)
plt.imshow(out_sr)
# print(tuple(iloc[0,1]),tuple(iloc[0,1]))


#crop
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.imshow(lr_c)
plt.title("Low-resolution")
plt.subplot(1,2,2)
plt.imshow(sr_c)
plt.title("Super-resolution")